<a href="https://colab.research.google.com/github/NiklasNesseler/SearchBasedSE/blob/main/uebung01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Aufgabe 1) Was ist die Idee von N-Grams in eigenen Worten?
- N-Grams teilen Text in Einheiten der Länge n auf
- werden überlappend gebildet
- Sinnvoll für LLMs
- werden verwendet um statistische Sprachmodelle zu erstellen
- Plagiaterkennung beispielsweise

## Aufgabe 2)

Gegeben ist folgender Minimaler Datensatz:

1. Sam is a cat
2. The cat is Sam
3. Is Sam a cat
4. That is Sam the cat

a) Stellen Sie eine Wahrscheinlichkeitsmatrix für ein n-gram mit n=2 auf. Beachten Sie
dabei keine Groß- und Kleinschrift.

Sam is, is a, a cat

The cat, cat is, is Sam

Is Sam, Sam a, a Cat

|  | is | a | cat | Sam |  |  |  |  |  |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| Sam | 0.5 | 0.5 | 0 | 0 |  |  |  |  |  |
| a | 0 | 0 | 1 | 0 |  |  |  |  |  |
| The | 0 | 0 | 1 | 0 |  |  |  |  |  |
| Cat | 1 | 0 | 0 | 0 |  |  |  |  |  |
| is | 0 | 0.33 | 0 | 0.67 |  |  |  |  |  |

b) Verwenden Sie dieses Model um für die folgenden Sätze zu bestimmen, welcher am
wahrscheinlichsten ist:
I. That is a cat
II. My cat is Sam
III. Is Sam the cat
IV. That is Sam

c) Verwenden Sie dieses Model um für die folgenden Sätze zu bestimmen, welches nächste
Wort am wahrscheinlichsten ist:
I. That is Sam
II. That is Sam is / a
III. The cat
IV. Sam is Sam

# Aufgaben
## Aufgabe 3: N-Grams in Python

Implementieren Sie entsprechend des Beispiels aus Aufgabe 2 ein Bigram Model in Python. Überprüfen Sie Ihre manuellen Erebnisse Ergebnisse.

## Aufgabe 4: Experimente mit größeren Daten

1. NLTK bietet verschiedene Text Corpora an. Testen Sie Ihre Bigram - Implementierung mit Jane Austen's Emma. Der Import steht in der letzten Zelle bereit. Installieren Sie nltk via `pip install nltk`.
2. Was wären Schritte, Preprocessing oder ähnliches, die Ihr Bigram verbessern könnten?

## Tokenization

In [26]:
data = ['Sam is a cat' ,
        'The cat is Sam' ,
        'Is Sam a cat' ,
        'That is Sam the cat']

In [27]:
import nltk
from nltk.corpus import gutenberg
# Download once if needed
nltk.download('gutenberg')
nltk.download('punkt')

# Choose a sample text
text = gutenberg.words('austen-emma.txt')  # Jane Austen's "Emma"
print(text)

['[', 'Emma', 'by', 'Jane', 'Austen', '1816', ']', ...]


[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [28]:
from collections import defaultdict

## Generate Bigrams

In [29]:
bigrams = defaultdict(int)
for sentence in data:
  words = sentence.lower().split()
  for i in range(len(words) - 1):
    bigram = (words[i], words[i + 1])
    bigrams[bigram] += 1

In [30]:
for bigram, count in bigrams.items():
  print(f'{bigram}: {count}')

('sam', 'is'): 1
('is', 'a'): 1
('a', 'cat'): 2
('the', 'cat'): 2
('cat', 'is'): 1
('is', 'sam'): 3
('sam', 'a'): 1
('that', 'is'): 1
('sam', 'the'): 1


## Generate a probability matrix

In [31]:
bigram_counts = defaultdict(int)
unigram_counts = defaultdict(int)

for sentence in data:
  words = sentence.lower().split()
  for i in range(len(words) - 1):
    w1, w2 = words[i], words[i + 1]
    bigram_counts[(w1, w2)] += 1
    unigram_counts[w1] += 1

In [32]:
prob_matrix = defaultdict(lambda: defaultdict(float))
for (w1, w2), count in bigram_counts.items():
  prob_matrix[w1][w2] = count / unigram_counts[w1]

for w1, probs in prob_matrix.items():
  for w2, prob in probs.items():
    print(f"P({w2} | {w1}) = {prob_matrix[w1][w2]:.2f}")

P(is | sam) = 0.33
P(a | sam) = 0.33
P(the | sam) = 0.33
P(a | is) = 0.25
P(sam | is) = 0.75
P(cat | a) = 1.00
P(cat | the) = 1.00
P(is | cat) = 1.00
P(is | that) = 1.00


## Calculate the probability of a sentence

In [33]:
def sentence_probability(sentence, prob_matrix):
  words = sentence.lower().split()
  prob = 1.0
  for i in range(len(words) - 1):
    w1, w2 = words[i], words[i + 1]
    if w2 in prob_matrix[w1]:
      prob *= prob_matrix[w1][w2]
    else:
      prob = 0.0
      break
  return prob

In [34]:
sentences = [
    'That is a cat',
    'My cat is Sam',
    'Is Sam the cat',
    'That is Sam'
]

In [35]:
for s in sentences:
  print(f'P({s}) = {sentence_probability(s, prob_matrix):.2f}')

P(That is a cat) = 0.25
P(My cat is Sam) = 0.00
P(Is Sam the cat) = 0.25
P(That is Sam) = 0.75


## Calculate the likeliest next word

In [36]:
def most_likely_next_word(phrase, prob_matrix):
  words = phrase.lower().split()
  if not words:
        # Alle möglichen Startwörter sammeln (flatten)
        all_candidates = {}
        for w1 in prob_matrix:
            for w2 in prob_matrix[w1]:
                if w1 not in all_candidates or prob_matrix[w1][w2] > all_candidates.get(w1, 0):
                    all_candidates[w1] = prob_matrix[w1][w2]
        if not all_candidates:
            return None
        return max(all_candidates, key=all_candidates.get)
  else:
        last_word = words[-1]
        candidates = prob_matrix.get(last_word, {})
        if not candidates:
            return None
        return max(candidates, key=candidates.get)

In [37]:
missing_words = [
    'That is',
    'That is Sam',
    'The',
    'Sam is',
    ''
]

In [38]:
for phrase in missing_words:
  next_word = most_likely_next_word(phrase, prob_matrix)
  print(f"Next word after '{phrase}': {next_word}")

Next word after 'That is': sam
Next word after 'That is Sam': is
Next word after 'The': cat
Next word after 'Sam is': sam
Next word after '': a


## Bonus: Generate the likeliest sentence.

In [39]:
sentence = ''
# TODO

# Aufgabe 4

In [44]:
import nltk
nltk.download('punkt_tab')
from nltk.corpus import gutenberg
from nltk.tokenize import sent_tokenize

# Download required resources
nltk.download('gutenberg')
nltk.download('punkt')

# Load raw text
raw_text = gutenberg.raw('austen-emma.txt')

# Split into sentences (remove if unwanted)
sentences_4 = sent_tokenize(raw_text)

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [45]:
import numpy as np
from collections import defaultdict, Counter
import pandas as pd

In [46]:
text = ' '.join(text)
text = text.lower()
import re
text = re.sub('[^a-z0-9\.\-]', ' ', text)
text = re.sub('\s{2,}', ' ', text)
text = '. ' + text.replace('.', ' .')
text = re.sub(' [a-z0-9] ', '', text)

In [47]:
print(text)

.  emma by jane austen 1816 volumechapteremma woodhouse handsome clever and rich withcomfortable home and happy disposition seemed to unite some of the best blessings of existence and had lived nearly twenty - one years in the world with very little to distress or vex her  . she was the youngest of the two daughters ofmost affectionate indulgent father and had in consequence of her sistermarriage been mistress of his house fromvery early period  . her mother had died too long ago for her to have more than an indistinct remembrance of her caresses and her place had been supplied by an excellent woman as governess who had fallen little short ofmother in affection  . sixteen years had miss taylor been in mr  . woodhousefamily less asgoverness thanfriend very fond of both daughters but particularly of emma  . between them it was more the intimacy of sisters  . even before miss taylor had ceased to hold the nominal office of governess the mildness of her temper had hardly allowed her to imp

In [48]:
words = text.split()
bgrams = {}
for i in range(len(words) - 2):
  w1 = words[i]
  w2 = words[i+1]
  if not w1 in bgrams:
    bgrams[w1] = {}

  if not w2 in bgrams[w1]:
    bgrams[w1][w2]= 1
  else:
    bgrams[w1][w2] += 1

In [49]:
for w in bgrams:
  bgrams[w] = dict(sorted(bgrams[w].items(), key=lambda item: -item[1]))
  total = sum(bgrams[w].values())
  bgrams[w] = dict([(k, bgrams[w][k]/total) for k in bgrams[w]])

In [50]:
print(bgrams)

{'.': {'she': 0.06257719226018937, 'weston': 0.05036366131466996, 'he': 0.049677507890764375, 'it': 0.04309043502127076, 'elton': 0.04199258954302182, 'the': 0.03952243721696171, 'knightley': 0.03691505420612049, 'you': 0.029916289282283518, 'and': 0.02621106079319336, 'but': 0.024838753945382187, 'emma': 0.02428983120625772, 'mr': 0.02387813915191437, 'they': 0.01729106628242075, 'woodhouse': 0.014683683271579526, 'there': 0.01399752984767394, 'oh': 0.013311376423768355, 'mrs': 0.012762453684643887, 'my': 0.011527377521613832, 'her': 0.011115685467270481, 'this': 0.010978454782489365, 'we': 0.01029230135858378, 'if': 0.009468917249897077, 'frank': 0.009057225195553726, 'no': 0.008645533141210375, 'harriet': 0.007684918347742555, 'churchill': 0.007135995608618087, 'how': 0.00699876492383697, 'perry': 0.00699876492383697, 'john': 0.00699876492383697, 'in': 0.006861534239055853, 'his': 0.006587072869493619, 'that': 0.006449842184712502, 'what': 0.006312611499931385, 'miss': 0.00617538081

In [51]:
import random

In [52]:
def next_word(word):
  vars = bgrams[word]
  return random.choices(list(vars.keys()), weights= vars.values())[0]

In [56]:
for i in range(5):
  print(next_word('book'))

--
--
again
it
you
